# Build an image classifier that tells if a fruit is a Durian or a Jackfruit
---
### Key Concepts:
  * Image Classifier (2 classes: {Durian, Jackfruit})
  * Transfer Learning (ResNet18)
  * Pytorch
  * ** <u>Local compute and local data</u> **

### Learning Objectives:
  * Load and explore Image dataset
  * Train model
  * Predict based on model (Inference) 

###  Dataset:
  * small dataset containing images of Durians and Jackfruits
---

In [ ]:
import torchvision
from torchvision import datasets, models, transforms
import os
import torch
import datetime
import time
import copy

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np

In [ ]:
# Setup variables
num_channels = 3        # RGB images = 3 channels
num_epochs = 2          # Number of Epochs to run the model through
learning_rate = 0.001   # Learning rate for the training
step_size=7             # Adjust learning rate schedule
gamma=0.1               #   step size details how many epochs before decaying LR by gamma 

data_dir = 'data'       # local folder where the images are stored
model_save_path = 'durian.pt' # local folder of the saved model



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## Load and Explore Data
---
1. Load Durian and Jackfruit images in local folder into train, validate and test datasets using Pytorch DataLoder's
1. Explore loaded data

In [ ]:
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([256, 256]),        
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize([256, 256]),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize([256, 256]),
        transforms.ToTensor(),
    ])   
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                        data_transforms[x])
                for x in ['train', 'val','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                            shuffle=True, num_workers=4)
            for x in ['train', 'val','test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}

class_names = image_datasets['train'].classes

print('Classes loaded:',class_names)

num_letters = len(class_names)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
# pull some samples from the validation dataset
images, labels = next(iter(dataloaders['val']))

n_images = len(labels)

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % class_names[labels[j]] for j in range(n_images))) 


## Train Classifier
---

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    # Save the model to a local file
    torch.save(model, model_save_path)

In [ ]:
print('Started training')

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

print('Number of classes:',len(class_names))
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=learning_rate, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=step_size, gamma=gamma)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=num_epochs)

print('Model trained.')

## Inference
---
1. Test a batch of images
1. Test a single image

In [ ]:
# functions to show an image
def imshow(img):
    npimg = img.cpu().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

correct = 0
total = 0

class_names = ['Durian', 'Jackfruit']

model = torch.load(model_save_path)
model.eval()

if torch.cuda.is_available():
    model.cuda()

with torch.no_grad():
    for i, (inputs, labels) in enumerate(dataloaders['test']):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs,1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()
        imshow(torchvision.utils.make_grid(inputs))
    
        print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]] for j in range(len(labels))))
        print('Predicted   : ', ' '.join('%5s' % class_names[preds[j]] for j in range(len(preds))))        
        
print('')
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total)) 

In [ ]:
import json
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline
plt.imshow(Image.open('data/test/Durian/images (2).jpg'))

In [ ]:
import torch
from torchvision import transforms
import torch
import torch.nn as nn
    
def preprocess(image_file):
    """Preprocess the input image."""
    data_transforms = transforms.Compose([
        transforms.Resize([256, 256]),
        transforms.ToTensor()
    ])

    image = Image.open(image_file)
    image = data_transforms(image).float()
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    return image

input_data = preprocess('data/test/Durian/images (2).jpg')

class_names = ['Durian', 'Jackfruit']

model = torch.load(model_save_path)
model.eval()

if torch.cuda.is_available():
    model.cuda()

with torch.no_grad():
    input_data = input_data.to(device)

    output = model(input_data)
    softmax = nn.Softmax(dim=1)
    pred_probs = softmax(output.cpu()).numpy()[0]
    index = torch.argmax(output, 1)
    result = {"label": class_names[index], "probability": str(pred_probs[index])}

print(result)